In [1]:
%matplotlib inline
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from typing import Optional, Tuple, Dict
import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
import pandas as pd
from MDAnalysis import transformations as mda_trans
from MDAnalysis.analysis import distances as mda_dist
import os
from polyphys.manage.typer import ParserT
from polyphys.manage.parser import SumRuleCyl, TransFociCyl, TransFociCub, Dump
from polyphys.manage.organizer import invalid_keyword, sort_filenames
from polyphys.analyze import clusters
from polyphys.analyze import correlations
from polyphys.probe import prober
from polyphys.probe.prober import stamps_report
import warnings

from MDAnalysis.analysis import polymer
from MDAnalysis.analysis import distances
from polyphys.manage.parser import HnsCub
from polyphys.manage import organizer

In [3]:
import MDAnalysis as mda
import MDAnalysis.analysis.msd as msd

### Probe explorations

In [ ]:
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
#path = "/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/N200epshm29kbmm10nh0ac2nc0l50dt0.005ndump2000adump5000ens1.ring"
#path = "/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/N200epshm29kbmm20nh48ac2nc0l60dt0.005ndump2000adump5000ens1.ring"
#path = "/Users/amirhsi_mini/research_data/N200epshm29kbmm10nh0ac2nc0l50dt0.005ndump2000adump5000ens1.ring"
#path = "../../Datasets/N200epshm29kbmm2nh48ac1nc95493l25dt0.005ndump2000adump5000ens1.ring"
#path = "/Users/amirhsi_mini/research_data/N200epshm29kbmm2nh0ac2nc0l25dt0.005ndump2000adump5000ens1.ring"
path = "../../N200epshm29kbmm2nh0ac2nc0l25dt0.005ndump2000adump5000ens1.ring"
nuc_pairs = glob(path + '/N*' + group + '*')
nuc_pairs = sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
nuc_pairs

In [ ]:
%%time
# 50 mins
for nuc_pair in nuc_pairs[:1]:
    topology = nuc_pair[0]
    trajectory = nuc_pair[1]
    lineage = 'whole'
    save_to = './'
    continuous = False
    if (lineage == 'segment') & (continuous is False):
        warnings.warn(
            "lineage is "
            f"'{lineage}' "
            "and 'continuous' is "
            f"'{continuous}. "
            "Please ensure the "
            f"'{trajectory}' is NOT part of a sequence of trajectories.",
            UserWarning
        )
    print("Setting the name of analyze file...")
    sim_info = HnsCub(
        trajectory,
        lineage,
        'cubic',
        'nucleoid',
        'ring'
    )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    print("\n" + sim_name + " is analyzing...\n")
    # LJ time difference between two consecutive frames:
    time_unit = sim_info.dmon * np.sqrt(
        sim_info.mmon * sim_info.eps_others)  # LJ time unit
    # Sampling via LAMMPS dump every 'ndump', so trajectory dt is:
    sim_real_dt = sim_info.ndump * sim_info.dt * time_unit
    cell = mda.Universe(
        topology, trajectory, topology_format='DATA',
        format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
        unwrap_images=True,
        atom_style="id resid type x y z", dt=sim_real_dt,
        )
    if continuous:
        sliced_trj = cell.trajectory[0: -1]
        n_frames = cell.trajectory.n_frames - 1
    else:
        sliced_trj = cell.trajectory[::10000]
        n_frames = cell.trajectory.n_frames
        n_frames = len(sliced_trj)
    # selecting atom groups:
    bug = cell.select_atoms('resid 1')  # the bug
    hns_patch = cell.select_atoms('type 2')  # the hns patches
    # transformations:
    workflow = [
        mda_trans.unwrap(cell.atoms),
        mda_trans.center_in_box(bug),
        mda_trans.wrap(cell.atoms)
    ]
    cell.trajectory.add_transformations(*workflow)
    # defining collectors
    # bug:
    gyr_t = []
    principal_axes_t = np.empty([0, 3, 3])
    asphericity_t = []
    shape_parameter_t = []
    # - bond info
    n_bonds = len(bug.bonds.indices)
    bond_lengths = np.zeros((n_bonds,1), dtype=np.float64)
    cosine_corrs = np.zeros(n_bonds, dtype=np.float64)
    # mon and hns-patch are attracted if their distance <= the below distance:
    m_hpatch_attr_cutoff = 0.5 * (sim_info.dmon + sim_info.dhns_patch)
    m_hpatch_shape = (sim_info.nmon, 2 * sim_info.nhns)
    m_m_shape = (sim_info.nmon, sim_info.nmon)
    # distance matrices
    dist_m_hpatch = np.zeros(m_hpatch_shape, dtype=np.float64)
    dist_m_m = np.zeros(m_m_shape, dtype=np.float64)
    # contact matrices
    dir_contacts_m_hpatch = np.zeros(m_hpatch_shape, dtype=np.int64)
    dir_contacts_m_m = np.zeros(m_m_shape, dtype=np.int64)
    for _ in sliced_trj:
        # bug:
        # -various measures of chain size
        gyr_t.append(bug.radius_of_gyration())
        # -shape parameters:
        asphericity_t.append(bug.asphericity())
        principal_axes_t = np.append(
            principal_axes_t,
            np.array([bug.principal_axes()]),
            axis=0
        )
        shape_parameter_t.append(bug.shape_parameter())
        # -bond info
        bond_dummy, cosine_dummy = correlations.bond_info(
            bug,
            sim_info.topology
            )
        bond_lengths += bond_dummy
        cosine_corrs += cosine_dummy
        # bug - hns patch:
        # - distance matrices
        dummy = mda_dist.distance_array(bug, hns_patch, box=cell.dimensions)
        print(dummy.shape)
        dist_m_hpatch += dummy
        dummy_m_m = np.matmul(dummy, dummy.T)
        dist_m_m += dummy_m_m
        # - contact matrices
        dummy = np.asarray(dummy <= m_hpatch_attr_cutoff, dtype=int)
        dir_contacts_m_hpatch += dummy
        dummy_m_m = np.matmul(dummy, dummy.T)
        dir_contacts_m_m += dummy_m_m
    # Saving collectors to memory
    # bug
    np.save(save_to + sim_name + '-gyrTMon.npy', np.array(gyr_t))
    outfile = save_to + sim_name + "-stamps.csv"
    stamps_report(outfile, sim_info, n_frames)
    np.save(save_to + sim_name + '-asphericityTMon.npy',
            np.array(asphericity_t)
            )
    np.save(save_to + sim_name + '-principalTMon.npy', principal_axes_t)
    np.save(save_to + sim_name + '-shapeTMon.npy', shape_parameter_t)
    bond_lengths = bond_lengths / n_frames
    bonds_per_lag = np.arange(n_bonds, 0, -1)
    cosine_corrs = cosine_corrs / (n_frames * bonds_per_lag)
    np.save(save_to + sim_name + '-bondLengthVecMon.npy', bond_lengths)
    np.save(save_to + sim_name + '-bondCosineCorrVecMon.npy', cosine_corrs)
    # bug hns-patch:
    dist_m_hpatch = dist_m_hpatch / n_frames
    dist_m_m = dist_m_m / n_frames
    dir_contacts_m_hpatch = dir_contacts_m_hpatch / n_frames
    dir_contacts_m_m = dir_contacts_m_m / n_frames
    np.save(
        save_to + sim_name + "-distMatTMonPatch.npy", dist_m_hpatch
    )
    np.save(
        save_to + sim_name + "-distMatTMonMon.npy", dist_m_m
    )
    np.save(
        save_to + sim_name + "-directContactsMatTMonPatch.npy",
        dir_contacts_m_hpatch
    )
    np.save(
        save_to + sim_name + "-directContactsMatTMonMon.npy",
        dir_contacts_m_m
    )
    # Simulation stamps:
    print('done.')

### MSD of monomers: g_1(t)

In [ ]:
MSD = msd.EinsteinMSD(cell, select='resid 1', msd_type='xyz', fft=True)
MSD.run()

In [ ]:
mda.__version__

In [ ]:
core_msd = MSD.results.timeseries
#core_msd

In [ ]:
nframes = MSD.n_frames
timestep = 1 # this needs to be the actual time between frames
lag_times = np.arange(nframes)*timestep # make the lag-time axis
fig = plt.figure()
ax = plt.axes()
# plot the actual MSD
ax.plot(lag_times, core_msd)#, lc="black", ls="-", label=r'3D random walk')
#exact = lag_times*6
# plot the exact result
#ax.plot(lag_times, exact, lc="black", ls="--", label=r'$y=2 D\tau$')

plt.show()

### g_2(t): mean-square-displacement in center of gravity of chain in the center of mass reference

In [ ]:
#time = []
rgyr = []
for ts in cell.trajectory:
    #if ts.time == 0:
    #if ts ==
    #time.append(cell.trajectory.time)
    rgyr.append(bug.radius_of_gyration())


In [ ]:
np.asarray(rgyr).mean()

In [ ]:
def gyr_squared(group, gyr_0, wrap=False):
    """Radius of gyration.
    Parameters
    ----------
    gyr_0
    group
    wrap : bool, optional
        If ``True``, move all atoms within the primary unit cell before
        calculation. [``False``]
    .. versionchanged:: 0.8
       Added `pbc` keyword
    .. versionchanged:: 2.1.0
       Renamed `pbc` kwarg to `wrap`. `pbc` is still accepted but
       is deprecated and will be removed in version 3.0.
    """
    atomgroup = group.atoms
    masses = atomgroup.masses
    com = atomgroup.center_of_mass(wrap=wrap)
    if wrap:
        recentered_pos = atomgroup.pack_into_box(inplace=False) - com
    else:
        recentered_pos = atomgroup.positions - com - gyr_0
    rog_sq = np.sum(masses * np.sum(recentered_pos**2,
                                    axis=1)) / atomgroup.total_mass()
    return rog_sq